<a href="https://colab.research.google.com/github/Dilipd0812/Objective-Learn-how-to-clean-and-prepare-raw-data-for-ML./blob/main/Breast_Cancer_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
uciml_breast_cancer_wisconsin_data_path = kagglehub.dataset_download('uciml/breast-cancer-wisconsin-data')

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import kagglehub
path = kagglehub.dataset_download("uciml/breast-cancer-wisconsin-data")
print(path)

In [ ]:
path

In [ ]:
try:
    files = os.listdir(path)
    for file in files:
        print(file)
except FileNotFoundError:
    print(f"Directory not found: {path}")

### LOAD DATA

In [ ]:
df = pd.read_csv(path + '/data.csv')
df.head()

### EDA

In [ ]:
df. shape

In [ ]:
# Look for nulls if present any
df.any().isnull()

In [ ]:
# Drop not required columns
df.drop('Unnamed: 32', axis=1, inplace=True)

In [ ]:
# check duplicates
df.duplicated().sum()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
# find unique value records to get the infromation on columns where LR can be performed
df.nunique()

In [ ]:
df['diagnosis'].value_counts()

In [ ]:
df = df.rename(columns={'diagnosis': 'target'})

In [ ]:
df.target.replace({'M': 1, 'B': 0}, inplace=True)

In [ ]:
df.target.value_counts()

In [ ]:
df.target = df.target.astype('float64')

In [ ]:
df.info()

In [ ]:
# check data stats
df.describe().T

### Analysis

In [ ]:
df.target.value_counts()

In [ ]:
df['target'].value_counts().plot(kind='bar',edgecolor='black',color=['lightsteelblue','navajowhite'])
plt.title(" Target",fontsize=20)
plt.show()

In [ ]:
cor = df.corr()
cor

In [ ]:
# SNS HEATMAP
plt.figure(figsize=(20,20))
sns.heatmap(cor, annot=True, cmap=plt.cm.CMRmap_r)
plt.show()

In [ ]:
threshold = 0.75
flr = np.abs(cor["target"] > threshold)
cf = cor.columns[flr].tolist()
cf

In [ ]:
# SNS Clustermap
sns.clustermap(df[cf].corr(), annot=True, cmap=plt.cm.CMRmap_r)
plt.show()

In [ ]:
# paiplot
sns.pairplot(df[cf], diag_kind = "kde" , markers = "*", hue="target")
plt.show()

### Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

In [ ]:
# split the data
x = df.drop('target', axis=1)
y = df['target']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
def met(model):
  model.fit(x_train, y_train)
  pred = model.predict(x_test)

  acc = accuracy_score(pred, y_test)
  cm = confusion_matrix(pred, y_test)
  ncm = confusion_matrix(pred, y_test, normalize='true')

  # Plot the confusion matrix
  print("Confusion Matrix")
  sns.heatmap(cm, annot=True, fmt='d',cmap=['lightsteelblue','navajowhite'])
  plt.title('Confusion matrix')
  plt.xlabel('Predcted lablel')
  plt.ylabel('True lable')
  plt.show()

  # Plot the Normalize Confusion Matrix
  print("Normalize Confusion Matrix")
  sns.heatmap(ncm, annot=True, fmt='.2f',cmap=['lightsteelblue','navajowhite'])
  plt.title('Normalize Confusion matrix')
  plt.xlabel('Predcted lablel')
  plt.ylabel('True lable')
  plt.show()

  print(cm)
  print(classification_report(pred, y_test))
  print("Accuracy Score: ", acc)

In [ ]:
model = LogisticRegression()
met(model)